Numan Kafadar 21946242 <br>
Umut Güngör 21946198
<h1  style="color:#005580"> Importing Necessary Liblaries and Reading Dataset</h1>

In [148]:
import copy
import numpy as np
import pandas as pd
import math
import time
import random
from  sklearn.model_selection import KFold 
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix

In [149]:
df=pd.read_csv("Employee-Attrition.csv")
dataset=df.copy()


<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#005580"> Part 1: Employee Attrition Prediction </h1>

<h1 style="font-size: 25px; font-weight: 600;text-transform:uppercase; color:#00264d"> Problem Definition and Solution Implementation </h1>

<ul>
  <li><span style="color:#cc3300">Problem Definition : </span>In this part, we are asked to implement a Decision Tree Model 
    on Employee dataset to classsify employee attrition value.</li> <br>
  <li><span style="color:#cc3300">Solution Steps : </span>Since some of the attribute values are continuous and Decision Tree performs better in discrete values, we first apply the discretization process on continuous attributes. After discretization process, we preare our training and testing sets with respect to the 5-fold cross validation. For each fold , we call ID3 algorithm for building tree and classify the test sets. After predicting test data, we examine our models performance using Classification Performance Metric values such as Accuracy, Precision, Recall and F1Score.
</li><br>
  <li><span style="color:#cc3300">Implementation Details : </span>
      For Discretization Process, we used Equal-Width Discretization method which simply extract minimum and maximum value of your continuous attributes and then create certain number of intervals
between range of minimum and maximum values for the discretization process. We decided our number of interval values as 3.
      <br>
      For ID3 algorithm, we need some helper functions to calculate information gain, total entropy and attribute entropy and decide maximum attribute to use which has highest information gain value. After implementing helper functions, we create our Node class which represents each Node of tree. To implement ID3 algorithm, we used the following pseudocode.<br><br>
  ID3(Examples, TargetAttribute, Attributes)<br>
• Create a Root node for the tree<br>
• Begin<br>
&ensp;– A <-- the attribute from Attributes that best classifies Examples<br>
&ensp;&ensp;– The decision attribute for Root <-- A<br>
&ensp;&ensp;– For each possible  value, vi, of A,<br>
&ensp;&ensp;&ensp;&ensp;• Add a new tree branch below Root, corresponding to the test A = vi<br>
&ensp;&ensp;&ensp;&ensp;• Let Examples_vibe the subset of Examples that have value vi for A<br>
&ensp;&ensp;&ensp;&ensp;• If Examples_vi is empty<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;– Then below this new branch add a leaf node with label = most common<br>
&ensp;&ensp;&ensp;&ensp;value of TargetAttribute in Examples<br>
&ensp;&ensp;&ensp;&ensp;• Else If Examples_vi target attribute contains only single value<br>
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;– Then below this new branch add a leaf node with label = most common<br>
&ensp;&ensp;&ensp;&ensp;value of TargetAttribute in Examples_vi<br>
&ensp;&ensp;&ensp;&ensp;•Else below this new branch add the subtree<br> 
&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;ID3(Examples_vi, TargetAttribute, Attributes – {A})<br>
• End<br>
• Return Root<br><br></li>
</ul>


In [150]:
#Equal-Width Discretization
def equal_width_discretization(arr,bins=3):
    #max and min values
    arr_max=max(arr)
    arr_min=min(arr)
    #weight 
    w = int((arr_max - arr_min) / bins)
    bin_slices = []
    #declaring intervals
    for i in range(0, bins + 1):
        if i==bins:
            bin_slices.append(arr_max)
        else:    
            bin_slices.append(arr_min + w * i)
    
    res=[0]*len(arr)
    
    #discretization of dataset column according to intervals
    for i in range(len(arr)):
        for j in range(len(bin_slices)-1):
            if arr[i] >= bin_slices[j] and arr[i] <= bin_slices[j+1]:
                res[i]=str(bin_slices[j])+" - "+str(bin_slices[j+1])
   
    return res


In [151]:
#discretization of dataset
columns_for_discretization=list(dataset.select_dtypes(include='number').columns)
for col in columns_for_discretization:
    dataset[col]=equal_width_discretization(dataset[col].to_numpy().tolist(),3)

#shuffling the dataset
dataset= dataset.sample(frac = 1)

In [152]:
#all Attributes Unique Values after Equal-Width Discretization
columnsUniqueValues={}
for i in dataset.columns:
    columnsUniqueValues[i]=list(dataset[i].unique())



In [153]:
#data exploring
dataset.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
752,32 - 46,Yes,Travel_Rarely,567 - 1032,Research & Development,10 - 19,3 - 5,Life Sciences,1 - 1,690 - 1379,...,3 - 4,80 - 80,0 - 1,13 - 26,0 - 2,3 - 4,13 - 26,12 - 18,10 - 15,10 - 17
174,32 - 46,No,Travel_Rarely,1032 - 1499,Sales,1 - 10,2 - 3,Life Sciences,1 - 1,1 - 690,...,1 - 2,80 - 80,1 - 2,0 - 13,2 - 4,3 - 4,0 - 13,0 - 6,0 - 5,0 - 5
494,32 - 46,No,Travel_Rarely,102 - 567,Sales,10 - 19,3 - 5,Technical Degree,1 - 1,1 - 690,...,3 - 4,80 - 80,2 - 3,0 - 13,2 - 4,3 - 4,0 - 13,0 - 6,0 - 5,5 - 10
178,46 - 60,No,Travel_Rarely,102 - 567,Sales,1 - 10,2 - 3,Marketing,1 - 1,1 - 690,...,3 - 4,80 - 80,2 - 3,13 - 26,2 - 4,3 - 4,13 - 26,12 - 18,10 - 15,5 - 10
821,46 - 60,No,Travel_Rarely,102 - 567,Sales,1 - 10,3 - 5,Technical Degree,1 - 1,690 - 1379,...,2 - 3,80 - 80,1 - 2,13 - 26,2 - 4,3 - 4,0 - 13,6 - 12,0 - 5,0 - 5


In [154]:
#total entropy of given dataset
def total_entropy(train_set,target_attribute="Attrition"):
    classes=train_set[target_attribute].unique() #"yes " or "no"
    entropy=0.0
    for c in classes:
        fraction = train_set[target_attribute].value_counts()[c]/len(train_set[target_attribute])
        entropy += -fraction * np.log2(fraction)

    return entropy

#attribute entropy of given dataset
def get_attribute_entropy(train_set,attribute, target_attribute="Attrition"):
    classes=train_set[target_attribute].unique() #"yes " or "no"
    variables = train_set[attribute].unique() #attribute classes
    total_entropy=0.0
    #for each value 
    for var in variables:
        entropy=0.0
        total_count = len(train_set[attribute][train_set[attribute]==var])
        #entropy of each attribute value
        for c in classes:
            value_count = len(train_set[attribute][train_set[attribute]==var][train_set[target_attribute] ==c])
            if total_count!=0 and value_count!=0:
                fraction = value_count/total_count
                entropy += -fraction*np.log2(fraction)
        total_fraction = total_count/len(train_set)
        total_entropy += total_fraction*entropy
    
    return total_entropy

#information gain is simply total_entropy-entropyofattribute
def get_information_gain(train_set,attribute,target_attribute="Attrition"):
    return total_entropy(train_set,target_attribute)-get_attribute_entropy(train_set,attribute,target_attribute)


#finds the attribute that has highest information gain
def find_max_attribute(train_set,attribute_list,target_attribute="Attrition"):
    ig_list=[]
    for attribute in attribute_list:
        ig=get_information_gain(train_set,attribute,target_attribute)
        ig_list.append(ig)
    i=np.argmax(ig_list) #index of attribute which has maximum information gain value
    
    return attribute_list[i]

#Tree Node for ID3 algroithm and Decision Tree 
class Node:
    def __init__(self):
        self.childs = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""
        self.parent=None

def ID3(train_set,columnsAttributeValues ,attribute_list, target_attribute="Attrition"):
    #create Root of Tree
    root=Node()
    #find the attribute that has maximum information gain
    max_attribute=find_max_attribute(train_set,attribute_list,target_attribute)
    root.value=max_attribute
    
   
    #for each value of max attribute
    for val in columnsAttributeValues[max_attribute]:
        #obtaining subdataset of max attribute value
        sub_train_set=train_set[train_set[max_attribute] == val]
        #Yes and No values
        target_vals=list(sub_train_set[target_attribute].unique())
        #creating new Node for child node
        newNode = Node()
        newNode.value = val
        newNode.parent=root
        
        #if subdataset is empty
        if len(sub_train_set)==0:
            
            #find most occured value in dataset and declare Node as leaf
            newNode.isLeaf = True
            target_vals=list(train_set[target_attribute].unique())
            
            maxPred=""
            maxPredCount=0
            for tval in target_vals:
                cnt=len(train_set[train_set[target_attribute]==tval])
                if cnt>maxPredCount:
                    maxPredCount=cnt
                    maxPred=tval
            newNode.pred = maxPred
            #add Node to the root
            root.childs.append(newNode)
            
        #if subdataset contains only "No" or "Yes" as Target Attribute
        elif len(target_vals)==1:
            #declare Node as leaf
            newNode.isLeaf = True
            newNode.pred = target_vals[0]
            root.childs.append(newNode)
        
        #else recursively call id3 algorithm
        else:
            #subdataset attributes list
            sub_attribute_list = attribute_list.copy()
            sub_attribute_list.remove(max_attribute)
            #getting child recursively and add it to the root
            child = ID3(sub_train_set,columnsAttributeValues,sub_attribute_list,target_attribute)
            child.parent=newNode
            newNode.childs.append(child)
            root.childs.append(newNode)
    
    #finally return root
    return root

#predicting the singe row with Decision tree        
def predict(root, test):
    #for each child
    for child in root.childs:
        
        if child.value == test[root.value]:
            #if child is leaf then return its pred value
            if child.isLeaf:
                
                return child.pred      
            #else recursively call predict
            else:
                return predict(child.childs[0], test)           
            
               
def printTree(root, depth=0):
    for i in range(depth):
        print("      ", end="")
    
    if len(root.childs)==1 or root.isLeaf:
        print(">"+root.value, end="")
    else:
        print("*"+root.value, end="")
    if root.isLeaf:
        print("->", root.pred)
    print()
    for child in root.childs:
        printTree(child, depth + 1)

In [155]:

#train and test datas
x=dataset.drop(columns=['Attrition']).to_numpy()
y=dataset['Attrition'].to_numpy()

columns=list(dataset.drop(columns=['Attrition']).columns)
columns.append('Attrition')

#Kfold-cross-validation
kf = KFold(n_splits=5)
kf.get_n_splits(x)

allFoldResults={"Fold 1":[],"Fold 2":[],"Fold 3":[],"Fold 4":[],"Fold 5":[]}
fnum=1
#best tree to print
bestTree=None
bestTreeAccuracy=0
for train_index, test_index in kf.split(x):
    #declaring x_test and y_test  as pd.Dataframe
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    train_data=pd.DataFrame(np.concatenate((x_train, np.asarray([y_train]).T), axis=1),columns=columns)
    x_test=pd.DataFrame(x_test,columns=list(dataset.drop(columns=['Attrition']).columns))
    
    st=time.time()
    attribute_list=list(train_data.drop(columns=["Attrition"]).columns)
    
    #call ID3 algoritm for training
    tree=ID3(train_data,columnsUniqueValues,attribute_list,"Attrition")
    tb=time.time()-st
    
    print("Training time for Fold {} is : {} sn".format(fnum,round(tb,3)))
    
    #Prediction Time
    preds=[]
    #for each row in x_test ,predict its value
    for index, row in x_test.iterrows():
        p=predict(tree,row)
        preds.append(p)
        
    #displaying accuracy,precision,recall and f1score metrics
    print("Fold {} Results \n  Accuracy :{} \n Precision :{} \n  Recall :{} \n  F1Score :{} \n\n".format(fnum,
            accuracy_score(y_test,preds),
            precision_score(y_test,preds,average='weighted'),
            recall_score(y_test,preds,average='weighted'),
            f1_score(y_test,preds,average='weighted')))
    
    #find best tree to print
    if accuracy_score(y_test,preds)>bestTreeAccuracy:
        bestTree=tree
        bestTreeAccuracy=accuracy_score(y_test,preds)
    
    
    allFoldResults[("Fold "+str(fnum))].append(accuracy_score(y_test,preds))
    allFoldResults[("Fold "+str(fnum))].append(precision_score(y_test,preds,average='weighted'))
    allFoldResults[("Fold "+str(fnum))].append(recall_score(y_test,preds,average='weighted'))
    allFoldResults[("Fold "+str(fnum))].append(f1_score(y_test,preds,average='weighted'))
                                        
    fnum+=1
    
    
    

Training time for Fold 1 is : 8.801 sn
Fold 1 Results 
  Accuracy :0.7755102040816326 
 Precision :0.7718149711430786 
  Recall :0.7755102040816326 
  F1Score :0.7736280487804877 


Training time for Fold 2 is : 8.954 sn
Fold 2 Results 
  Accuracy :0.8129251700680272 
 Precision :0.8034193170379941 
  Recall :0.8129251700680272 
  F1Score :0.807923915630762 


Training time for Fold 3 is : 9.801 sn
Fold 3 Results 
  Accuracy :0.7687074829931972 
 Precision :0.794770111912298 
  Recall :0.7687074829931972 
  F1Score :0.780661614288234 


Training time for Fold 4 is : 9.228 sn
Fold 4 Results 
  Accuracy :0.7687074829931972 
 Precision :0.7374351056211959 
  Recall :0.7687074829931972 
  F1Score :0.7493155798904927 


Training time for Fold 5 is : 8.719 sn
Fold 5 Results 
  Accuracy :0.7755102040816326 
 Precision :0.7831115985488847 
  Recall :0.7755102040816326 
  F1Score :0.7791753436068305 




<h1 style="font-size: 25px; font-weight: 600;text-transform:uppercase; color:#00264d"> Tree that has highest accuracy in 5-fold </h1>

In [156]:
printTree(bestTree)

*JobRole
      >Laboratory Technician
            *OverTime
                  >No
                        *MaritalStatus
                              >Single
                                    *YearsWithCurrManager
                                          >10 - 17-> Yes

                                          >0 - 5
                                                *Age
                                                      >32 - 46
                                                            *RelationshipSatisfaction
                                                                  >3 - 4-> No

                                                                  >1 - 2
                                                                        *EducationField
                                                                              >Life Sciences-> Yes

                                                                              >Technical Degree-> No

                                

                                                                  >567 - 1032-> No

                                                                  >1032 - 1499-> No

                                                                  >102 - 567-> Yes

                                          >1379 - 2068-> No

      >Sales Representative
            *BusinessTravel
                  >Travel_Rarely
                        *RelationshipSatisfaction
                              >3 - 4
                                    *DailyRate
                                          >567 - 1032
                                                *DistanceFromHome
                                                      >10 - 19-> No

                                                      >1 - 10-> Yes

                                                      >19 - 29
                                                            *Education
                                                                  >3 - 

In [158]:
allFolds=pd.DataFrame.from_dict(allFoldResults,orient='index',
                       columns=['Accuracy', 'Precision', 'Recall', 'F1Score'])
allFolds.loc['Avrg. OF Folds'] = allFolds.mean()

#prints table as Jupter Markdown
#print(allFolds.to_markdown())

<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#005580"> Part 1: Error and Performance Analysis for Classification </h1>
<h1 style="font-size: 25px; font-weight: 600;text-transform:uppercase; color:#00264d;text-align:center"> Classification Performance Metric  Table</h1>

|                |   Accuracy |   Precision |   Recall |   F1Score |
|:---------------|-----------:|------------:|---------:|----------:|
| Fold 1         |   0.77551  |    0.771815 | 0.77551  |  0.773628 |
| Fold 2         |   0.812925 |    0.803419 | 0.812925 |  0.807924 |
| Fold 3         |   0.768707 |    0.79477  | 0.768707 |  0.780662 |
| Fold 4         |   0.768707 |    0.737435 | 0.768707 |  0.749316 |
| Fold 5         |   0.77551  |    0.783112 | 0.77551  |  0.779175 |
| Avrg. OF Folds |   0.780272 |    0.77811  | 0.780272 |  0.778141 |


<ul>
      <li><span style="color:#cc3300">Misclassified Samples: </span>When ID3 algorithm is run, it is noticed that there are some examples that are misclassified. One reason that causes this misclassification can be the overfitting problem. If the decision tree model is very long, accuracy will be decrease. Therefore there are some examples that are misclassified. When we look at some spesific misclassified samples, we see that these samples' continuous attributes cause this misclassification. After discretization process, some of the samples are hard to classify correctly.</li>
  <li><span style="color:#cc3300">Decision Tree Performance: </span>We apply 5-fold cross validation to test the performance of the model. The performance metrics such as accuracy, precision, recall and F1 score are around 80%. That can be seen from the table above. Since these metrics are around 80%, we can say that the performance of this model is not very good. However, since we apply 5-fold cross validation, results of some folds are better than others. For example, in fold 2, the accuracy, recall and F1 scores are the highest values out of 5 folds. But, the highest value for precision metric is in fold 5. So, we can say that, these metrics are not always proportional. When one of them increases, another can be decrease. Also, computation time is different for all folds. Because, when we split the data into training and test parts in different ways the model changes. Using different training and test data sets affect computation time. For fold 3, training lasts approximately 9.8 seconds and this is the longest time in 5 folds.</li>
</ul>

<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#005580"> Part 2: Pruning Decision Tree </h1>

<h1 style="font-size: 25px; font-weight: 600;text-transform:uppercase; color:#00264d"> Problem Definition and Solution Implementation </h1>

<ul>
  <li><span style="color:#cc3300">Problem Definition : </span>We need to prevent overfit of our decision tree by pruning the twigs of our tree.The twigs are the nodes whose children are all leaves. </li><br>
  <li><span style="color:#cc3300">Solution Steps : </span>For prunning, we split our dataset into 5-folds again and each fold has %60 of data as train set, %20 of data as validation set and %20 of data as test set. We prunn our tree on validation set and then test our model on test set.</li><br>
  <li><span style="color:#cc3300">Implementation Details : </span>For prunning , we used the following pseudocode for implementation: <br>
    
– Step 1: Create an ”Last Accuracy” variable and set this accuracy to the accuracy of your <br>
decision tree model on validation setCatalog all twigs in the tree<br>
– Step 2: Find the twig with the least Information Gain<br>
– Step 3: Remove all child nodes of the twig<br>
– Step 4: Relabel twig as a leaf (Set the majority of ”Positive” or ”Negative”
as leaf value)<br>
– Step 5: Measure the accuracy value of your decision tree model with removed<br>
twig on the validation set (”Current Accuracy”)<br>
– If ”Current Accuracy ≥ Last Accuracy” : Jump to ”Step1”<br>
Else : Revert the last changes done in Step 3,4 and then terminate<br></li>
</ul>

In [159]:
#return if node is leaf
def isLeaf(node):   
    return node.isLeaf

#return if node is twig
def isTwig(node):
    for child in node.childs:
        if not isLeaf(child):
            return False
    return True

#get all twigs
def getAllTwigs(node,twlist):
    #if node is twig ,add to twigList and return
    if isTwig(node):
        twlist.append(node)
        return
    else: 
        #else ,recursively call it for each child
        for child in node.childs:
            if len(child.childs)>0:
                getAllTwigs(child.childs[0],twlist)
    return twlist

#method for obtaining subdataset of twig
#recursion from twig to root
def getTwigDataset(root,dataset,twig):
    if root==twig:
        return dataset 
    else: 
        if twig is not None:
            return getTwigDataset(root,dataset[dataset[twig.parent.parent.value] == twig.parent.value],twig.parent.parent)
        


In [160]:


def prunTree(train_data , y_validation, y_test, targetAttribute="Attrition"):
    
    st=time.time()
    
    attribute_list=list(train_data.drop(columns=["Attrition"]).columns)
    
    #obtain tree 
    tree=ID3(train_data,columnsUniqueValues,attribute_list,"Attrition")
    treeBeforePrun=copy.deepcopy(tree)
    
    #calculate accuracy on test data before prunning
    preds=[]
    for index, row in x_test.iterrows():    
        p=predict(tree,row) 
        preds.append(p)
        
    accuracyWithoutPrunning=accuracy_score(y_test,preds)
    print("Accuracy Without Prunning -Test Set : {}".format(accuracyWithoutPrunning))
    
    print("\nPrunning Process begins...\n")
    #calculate accuracy on validation data before prunning
    preds=[]
    for index, row in x_validation.iterrows():
        p=predict(tree,row) 
        preds.append(p)
    accuracyBeforePrunning=accuracy_score(y_validation,preds)
    
    prunCount=0
    while True:
        
        print("Accuracy Before Prunning -Validation Set: {}".format(accuracyBeforePrunning))
        
        minInfoGainTwig=""
        minInfoGain=10
        #all twigs
        twigList=getAllTwigs(tree,[])
        #find the twig that has least information gain
        for twig in twigList:
            subdataForTwig=getTwigDataset(tree,dataset,twig)
            ig=get_information_gain(subdataForTwig,twig.value)
            if ig<minInfoGain:
                minInfoGain=ig
                minInfoGainTwig=twig

        
        prunSet=[]
        #remove from twiglist
        twigList.remove(minInfoGainTwig)
        
        #get predictions of twig
        for child in minInfoGainTwig.childs:
            prunSet.append(child.pred)
       
        #create new leaf node
        newNode=Node()
        newNode.isLeaf=True
        newNode.value=minInfoGainTwig.parent.value
        #declare its prediction value as max of child values
        newNode.pred=max(set(prunSet), key=prunSet.count)
        
        #prunning the tree
        prunIndex=0 #this value is used later on reversing prunning
        for pchild in range(len(minInfoGainTwig.parent.parent.childs)):
            if minInfoGainTwig.parent.parent.childs[pchild].value==minInfoGainTwig.parent.value:
                minInfoGainTwig.parent.parent.childs[pchild]=newNode
                prunIndex=pchild
                break
               
        #calculate accuracy after prunn on validation data
        preds=[]
        for index, row in x_validation.iterrows():   
            p=predict(tree,row) 
            preds.append(p)
                    
        accuracyAfterPrunning=accuracy_score(y_validation,preds)
        print("Accuracy After Prunning -Validation Set: {}".format(accuracyAfterPrunning))
        
        #if accuracy incerased or remained same, do the same operations again
        if(accuracyAfterPrunning>=accuracyBeforePrunning):
            accuracyBeforePrunning=accuracyAfterPrunning
            prunCount+=1
        
        #if accuracy is decreased
        else:
            #reverse last prunning 
            print("Accuracy decreased, reversing last prunn..")
            minInfoGainTwig.parent.parent.childs[prunIndex]=minInfoGainTwig.parent
            break
        
    print("Prunned {} times.".format(prunCount))
    tb=time.time()-st
    print("Prunning Process completed in {} seconds".format(tb))
    
    #calculate accuracy on test data after prunning is completed
    preds=[]
    for index, row in x_test.iterrows():
        p=predict(tree,row) 
        preds.append(p)
        
    accuracyWithPrunning=accuracy_score(y_test,preds)
    
  
            
    print("\nAccuracy With Prunning -Test Set: {}".format(accuracyWithPrunning))
    #return accuracy before and after prunning on test data
    return accuracyWithoutPrunning,accuracyWithPrunning,treeBeforePrun,tree
  
        

In [161]:

#for declaring best tree in 5-fold that has highest accuracy gain after prunning 
bestTreeDifOfPrun=0
bestTreeBeforePrun=None
bestTreeAfterPrun=None

fnum=1
resultOfPrunning={"Fold 1":[],"Fold 2":[],"Fold 3":[],"Fold 4":[],"Fold 5":[]}
for train_index, test_index in kf.split(x):
    #declaring x_test and y_test  as pd.Dataframe
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #train data -%80 of dataset
    train_data=pd.DataFrame(np.concatenate((x_train, np.asarray([y_train]).T), axis=1),columns=columns)
    #test_data -%20 of dataset
    x_test=pd.DataFrame(x_test,columns=list(dataset.drop(columns=['Attrition']).columns))
    
    attribute_list=list(train_data.drop(columns=["Attrition"]).columns)
    
    #validation data -%25 of train data (%20 of total dataset)
    validation_data=train_data[882:1176]
    #train data decreased to %60 of dataset
    train_data=train_data[:882]
    #y_validation and y_test
    y_validation=validation_data["Attrition"]
    x_validation=pd.DataFrame(validation_data,columns=list(dataset.drop(columns=['Attrition']).columns))
    
    print("\nFold {} -Prunning on Validation Data....".format(fnum))
    
    #calling prunning tree, aap is accuracy before prunning and aap accuracy after prunning
    abp,aap,treeBeforePrun,treeAfterPrun=prunTree(train_data,y_validation,y_test,targetAttribute="Attrition")
    
    #find the best tree that highest accuracy change 
    if (aap-abp) > bestTreeDifOfPrun:
        bestTreeDifOfPrun=aap-abp
        bestTreeBeforePrun=treeBeforePrun
        bestTreeAfterPrun=treeAfterPrun
    
    resultOfPrunning["Fold "+str(fnum)]=[abp,aap]
    fnum+=1


Fold 1 -Prunning on Validation Data....
Accuracy Without Prunning -Test Set : 0.7925170068027211

Prunning Process begins...

Accuracy Before Prunning -Validation Set: 0.7755102040816326
Accuracy After Prunning -Validation Set: 0.7789115646258503
Accuracy Before Prunning -Validation Set: 0.7789115646258503
Accuracy After Prunning -Validation Set: 0.7789115646258503
Accuracy Before Prunning -Validation Set: 0.7789115646258503
Accuracy After Prunning -Validation Set: 0.7891156462585034
Accuracy Before Prunning -Validation Set: 0.7891156462585034
Accuracy After Prunning -Validation Set: 0.7891156462585034
Accuracy Before Prunning -Validation Set: 0.7891156462585034
Accuracy After Prunning -Validation Set: 0.7925170068027211
Accuracy Before Prunning -Validation Set: 0.7925170068027211
Accuracy After Prunning -Validation Set: 0.7925170068027211
Accuracy Before Prunning -Validation Set: 0.7925170068027211
Accuracy After Prunning -Validation Set: 0.7925170068027211
Accuracy Before Prunning -

Accuracy After Prunning -Validation Set: 0.7993197278911565
Accuracy Before Prunning -Validation Set: 0.7993197278911565
Accuracy After Prunning -Validation Set: 0.7959183673469388
Accuracy decreased, reversing last prunn..
Prunned 30 times.
Prunning Process completed in 11.786883354187012 seconds

Accuracy With Prunning -Test Set: 0.8163265306122449

Fold 5 -Prunning on Validation Data....
Accuracy Without Prunning -Test Set : 0.7517006802721088

Prunning Process begins...

Accuracy Before Prunning -Validation Set: 0.8061224489795918
Accuracy After Prunning -Validation Set: 0.8061224489795918
Accuracy Before Prunning -Validation Set: 0.8061224489795918
Accuracy After Prunning -Validation Set: 0.8061224489795918
Accuracy Before Prunning -Validation Set: 0.8061224489795918
Accuracy After Prunning -Validation Set: 0.8061224489795918
Accuracy Before Prunning -Validation Set: 0.8061224489795918
Accuracy After Prunning -Validation Set: 0.8095238095238095
Accuracy Before Prunning -Validation

<h1 style="font-size: 25px; font-weight: 600;text-transform:uppercase; color:#00264d"> printing Tree that has highest accuracy gain before and after prunning    </h1>

To see the twigs that prunned in tree: https://www.diffchecker.com/uwvb1s61

In [167]:
#before prunning
printTree(bestTreeBeforePrun)

*JobRole
      >Laboratory Technician
            *OverTime
                  >No
                        *YearsInCurrentRole
                              >12 - 18
                                    *DailyRate
                                          >567 - 1032-> Yes

                                          >1032 - 1499-> No

                                          >102 - 567-> Yes

                              >0 - 6
                                    *RelationshipSatisfaction
                                          >3 - 4
                                                *JobSatisfaction
                                                      >1 - 2
                                                            *MonthlyRate
                                                                  >2094 - 10395-> Yes

                                                                  >10395 - 18696
                                                                        *Age
              

                                                                                          >46 - 60-> Yes

                                                                                          >18 - 32-> Yes

                                                                              >19 - 29-> Yes

                                                      >2 - 3-> No

                                                      >1 - 2
                                                            *Age
                                                                  >32 - 46-> No

                                                                  >46 - 60-> Yes

                                                                  >18 - 32-> Yes

                                          >10395 - 18696
                                                *EducationField
                                                      >Life Sciences-> No

                                                      >Techn

In [168]:
#after prunning
printTree(bestTreeAfterPrun)

*JobRole
      >Laboratory Technician
            *OverTime
                  >No
                        *YearsInCurrentRole
                              >12 - 18
                                    *DailyRate
                                          >567 - 1032-> Yes

                                          >1032 - 1499-> No

                                          >102 - 567-> Yes

                              >0 - 6
                                    *RelationshipSatisfaction
                                          >3 - 4
                                                *JobSatisfaction
                                                      >1 - 2
                                                            *MonthlyRate
                                                                  >2094 - 10395-> Yes

                                                                  >10395 - 18696
                                                                        *Age
              

                                          >46 - 60-> Yes

                                          >18 - 32-> Yes

                              >1 - 10-> No

                              >19 - 29-> Yes

      >Healthcare Representative
            *EnvironmentSatisfaction
                  >3 - 4-> No

                  >2 - 3-> No

                  >1 - 2
                        *MonthlyRate
                              >2094 - 10395-> No

                              >10395 - 18696
                                    *MaritalStatus
                                          >Single-> No

                                          >Divorced-> Yes

                                          >Married-> Yes

                              >18696 - 26999-> No

      >Manufacturing Director
            *RelationshipSatisfaction
                  >3 - 4
                        *MonthlyRate
                              >2094 - 10395-> No

                              >10395 - 18696-> No


In [164]:
allFoldsforPrunning=pd.DataFrame.from_dict(resultOfPrunning,orient='index',
                       columns=['Accuracy Before Prunning','Accuracy After Prunning'])
allFoldsforPrunning.loc['Avrg. OF Folds'] = allFoldsforPrunning.mean()
#printing Table as Juptre Markdown
#print(allFoldsforPrunning.to_markdown())

<h1 style="font-size: 40px; font-weight: 600;text-transform:uppercase; color:#005580"> Part 2: Error and Performance Analysis for Prunning </h1>

<h1 style="font-size: 25px; font-weight: 600;text-transform:uppercase; color:#00264d;text-align:center"> Accuracy Table Of Prunning -Test Data</h1>

|                |   Accuracy Before Prunning |   Accuracy After Prunning |
|:---------------|---------------------------:|--------------------------:|
| Fold 1         |                   0.792517 |                  0.806122 |
| Fold 2         |                   0.795918 |                  0.802721 |
| Fold 3         |                   0.826531 |                  0.85034  |
| Fold 4         |                   0.806122 |                  0.816327 |
| Fold 5         |                   0.751701 |                  0.795918 |
| Avrg. OF Folds |                   0.794558 |                  0.814286 |

<ul>
    <li><span style="color:#cc3300">Reason of Prunning: </span>Decision trees that are trained on any training data run the risk of overfitting the training data. What we mean by this is that eventually each leaf will reperesent a very specific set of attribute combinations that are seen in the training data, and the tree will consequently not be able to classify attribute value combinations that are not seen in the training data.
In order to prevent this , we must prune the twigs of our decision tree.</li>
  <li><span style="color:#cc3300">Analysis of Prunning: </span>Aim of the prunning is remove redundant attributes from the decision tree. So, in each 5 fold, accuracy of the model increases after the prunning process is completed. It can be seen form the table above. Some of the folds, accuracy value after prunning increases more. For instance, in fold 5, increasing value of the accuracy is around 4%. Different distribution of the data into training and test sets may affect this diversities. Computation time for prunning is around 10 seconds. It is a bit longer than training the model. However, it is worth to do it beacuse it increases the performance of the model.</li>
</ul>